# Data Importing and Saving

In [1]:
import numpy as np
import pandas as pd

In [2]:
from pathlib import Path
# Sets a variable to the home directory of the current user
home = str(Path.home())

## JSON Data

In [3]:
# Manually defined json-like object as a list of dicts
nobel_winners = [
        {
            "category": "Physics",
            "name": "Albert Einstein",
            "nationality": "Swiss",
            "sex": "male",
            "year": "1921"
        },
        {
            "category": "Physics",
            "name": "Paul Dirac",
            "nationality": "British",
            "sex": "male",
            "year": "1933"
        },
        {
            "category": "Chemistry",
            "name": "Marie Curie",
            "nationality": "Polish",
            "sex": "female",
            "year": "1911"
        }
    ]


### Converting JSON Data to CSV

In [4]:
# Create headers from json object (assumes consistent schema)
cols = nobel_winners[0].keys()
columns = sorted(cols)

winners_path = f'{home}/data/nobel_winners.csv'

# Open System file
with open(winners_path, 'w') as f:
    f.write(','.join(columns) + '\n')
    for o in nobel_winners:
        row = [str(o[col]) for col in columns]
        f.write(','.join(row) + '\n')


In [5]:
# Here we are going to use the context manager with ... to verify that we wrote the file properly
with open(winners_path) as f:
    for line in f.readlines():
        print(line)

category,name,nationality,sex,year

Physics,Albert Einstein,Swiss,male,1921

Physics,Paul Dirac,British,male,1933

Chemistry,Marie Curie,Polish,female,1911



### Using Pandas to open a csv and to specify delimiter

In [6]:
# Let's open the file we wrote again, but this time using Pandas
content = pd.read_csv(winners_path)

In [7]:
type(content)

pandas.core.frame.DataFrame

In [8]:
# Displaying the content that has been saved in a pandas DataFrame
content

,category,name,nationality,sex,year
0,Physics,Albert Einstein,Swiss,male,1921
1,Physics,Paul Dirac,British,male,1933
2,Chemistry,Marie Curie,Polish,female,1911


We're going to write a new version of the file, but with a semicolon delimiter this time

In [9]:
# Setting a variable for the path of the new file
winners_path1 = f'{home}/data/nobel_winners1.csv'
with open(winners_path1, 'w') as f:
    f.write(';'.join(columns) + '\n')
    for o in nobel_winners:
        row = [str(o[col]) for col in columns]
        f.write(';'.join(row) + '\n')

When we save the new content to a new DataFrame without specifying the delimiter, we can see that the parsing wasn't right

In [10]:
content_no_delimiter_specified = pd.read_csv(winners_path1)

content_no_delimiter_specified

,category;name;nationality;sex;year
0,Physics;Albert Einstein;Swiss;male;1921
1,Physics;Paul Dirac;British;male;1933
2,Chemistry;Marie Curie;Polish;female;1911


To fix the parsing, now we are going to specify the delimiter to use

In [11]:
content_with_delimiter_specified = pd.read_csv(winners_path1, sep =';')
content_with_delimiter_specified

,category,name,nationality,sex,year
0,Physics,Albert Einstein,Swiss,male,1921
1,Physics,Paul Dirac,British,male,1933
2,Chemistry,Marie Curie,Polish,female,1911


Especially useful with big data sets, we can display the first few rows of data.


Optionally, we can specify how many to show at once with an integer.


In [12]:
content.head(2)

,category,name,nationality,sex,year
0,Physics,Albert Einstein,Swiss,male,1921
1,Physics,Paul Dirac,British,male,1933


### Updating columns in the DataFrame

Let's specify a column that we want to change by putting it into a python dict

In [13]:
updatedColumns = {"category":"field"}

Here we will rename a column

This is reminiscent of React's method of merging state changes, rather than overwriting all state.
Similar behavior is also found in Git's patching.

In [14]:
content = content.rename(columns=updatedColumns)
content

,field,name,nationality,sex,year
0,Physics,Albert Einstein,Swiss,male,1921
1,Physics,Paul Dirac,British,male,1933
2,Chemistry,Marie Curie,Polish,female,1911


### Convert DataFrame to Dictionary

DataFrames can be converted to Python objects

We will use this below in the JSON section

In [15]:
converted_records = content.to_dict(orient="records")

### Queries on Pandas DataFrames

We can also perform queries on the data, similarly to using where clauses in SQL

In [16]:
physicists = content[content.field == "Physics"]

physicists

,field,name,nationality,sex,year
0,Physics,Albert Einstein,Swiss,male,1921
1,Physics,Paul Dirac,British,male,1933


Similarly, we can view a single column of data

In [17]:
sexes_of_winners = content.sex
sexes_of_winners

0      male
1      male
2    female
Name: sex, dtype: object

### Using loc to perform queries with filters
See this website for more information:
[Ritchieng - Pandas Selecting Multiple Rows and Columns](https://www.ritchieng.com/pandas-selecting-multiple-rows-and-columns/)

#### Query with a filter and selecting a single column

In [18]:
sex_of_winners_after_1920 = content.loc[content.year >= 1920, ['sex']]
sex_of_winners_after_1920

,sex
0,male
1,male


#### Query with multiple columns

In [19]:
nationality_and_year = content.loc[(content.year >= 1920), ['nationality', 'year']]
nationality_and_year

,nationality,year
0,Swiss,1921
1,British,1933


#### Querying with multiple filters and multiple columns

In [20]:
nationality_and_year_female = content.loc[(content.year >= 1920) & (content.sex != 'male'), ['nationality', 'year']]
nationality_and_year_female

,nationality,year


## CSV Module

In [21]:
import csv

Here is the nobel_winners dict that we are working with:

In [22]:
nobel_winners

[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': '1921'},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': '1933'},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': '1911'}]

### Writing a new csv file with the built-in csv module

In [23]:
winners_path2 = f'{home}/data/nobel_winners2.csv'

with open(winners_path2, 'w') as f:
    fieldnames = nobel_winners[0].keys()
    sorted_fieldnames = sorted(fieldnames)
    writer = csv.DictWriter(f, fieldnames=sorted_fieldnames)
    writer.writeheader()
    for r in nobel_winners:
        writer.writerow(r)


In [24]:
with open(winners_path2, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        print(row)

OrderedDict([('category', 'Physics'), ('name', 'Albert Einstein'), ('nationality', 'Swiss'), ('sex', 'male'), ('year', '1921')])
OrderedDict([('category', 'Physics'), ('name', 'Paul Dirac'), ('nationality', 'British'), ('sex', 'male'), ('year', '1933')])
OrderedDict([('category', 'Chemistry'), ('name', 'Marie Curie'), ('nationality', 'Polish'), ('sex', 'female'), ('year', '1911')])


Another example of retrieving items from the csv, this time appending them to a list

In [25]:
years = []
with open(winners_path2, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        years.append(row['year'])
years

['1921', '1933', '1911']

### Parsing fields returned
As you can see above, the values for the years that we pull from the csv file with the reader are not parsed.
We need to parse these manually to use them as ints

In [26]:
with open(winners_path2, 'r') as f:
    reader = csv.DictReader(f)
    winners = list(reader)
    for winner in winners:
        winner['year'] = int(winner['year'])

Let's take the average of the years sampled to show that they are numbers

In [27]:
sumYears = 0
for w in winners:
    sumYears += w['year']
average = round(sumYears/len(winners))
average

1922

## Working with JSON

In [28]:
import json

### Writing and Reading JSON Data

In [29]:
winners_json = f'{home}/data/nobel_winners.json'

with open(winners_json, 'w') as f:
    json.dump(nobel_winners, f)

with open(winners_json, 'r') as f:
    nobels = json.load(f)

nobels

[{'category': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': '1921'},
 {'category': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': '1933'},
 {'category': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': '1911'}]

Here we will use the converted_records object from the pandas examples above.

There we changed one of the column headers from the original json.

Then we converted the object to a python dict to save here using the to_dict method.

In [30]:
with open(winners_json, 'w') as f:
    json.dump(converted_records, f)

with open(winners_json, 'r') as f:
    nobels2 = json.load(f)

nobels2

[{'field': 'Physics',
  'name': 'Albert Einstein',
  'nationality': 'Swiss',
  'sex': 'male',
  'year': 1921},
 {'field': 'Physics',
  'name': 'Paul Dirac',
  'nationality': 'British',
  'sex': 'male',
  'year': 1933},
 {'field': 'Chemistry',
  'name': 'Marie Curie',
  'nationality': 'Polish',
  'sex': 'female',
  'year': 1911}]